# LangGraph Tutorial – Part 3️⃣: Agents và Tools

**Mục tiêu học tập:** Tác nhân biết sử dụng công cụ

Trong phần này, chúng ta sẽ tìm hiểu:
- Cách tích hợp Agents với Tools trong LangGraph
- Xây dựng agent có khả năng tìm kiếm web
- Sử dụng Conditional Edges để agent quyết định hành động
- Phân tích quy trình lựa chọn công cụ

## 🔧 Giới thiệu: Agents, Tools trong LangGraph

**Agent** là một thực thể AI có khả năng:
- Nhận thông tin từ môi trường
- Quyết định hành động dựa trên thông tin đó
- Sử dụng các công cụ (Tools) để thực hiện hành động
- Học hỏi từ kết quả để cải thiện quyết định

**Tools** là các chức năng mà Agent có thể sử dụng:
- Tìm kiếm web
- Tính toán
- Truy cập database
- API calls
- File operations

![Agent-Tool Architecture](https://python.langgraph.org/img/agent_executor.png)

## 📦 Cài đặt & Cấu hình

Đầu tiên, chúng ta cần cài đặt các thư viện cần thiết:

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install langgraph langchain-openai langchain-anthropic duckduckgo-search python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from duckduckgo_search import DDGS

# Load environment variables
load_dotenv()

# Khởi tạo LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

## 🛠️ Định nghĩa Tools

Tạo công cụ tìm kiếm web sử dụng DuckDuckGo:

In [ ]:
@tool
def web_search(query: str) -> str:
    """Tìm kiếm thông tin trên web bằng DuckDuckGo.
    
    Args:
        query: Từ khóa tìm kiếm
        
    Returns:
        Kết quả tìm kiếm dưới dạng text
    """
    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=3))
            
        if not results:
            return f"Không tìm thấy kết quả cho: {query}"
        
        # Format kết quả
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. **{result['title']}**\n"
                f"   {result['body']}\n"
                f"   🔗 {result['href']}\n"
            )
            
        return f"🔍 Kết quả tìm kiếm cho '{query}':\n\n" + "\n".join(formatted_results)
        
    except Exception as e:
        return f"Lỗi khi tìm kiếm: {str(e)}"

@tool 
def calculator(expression: str) -> str:
    """Tính toán biểu thức toán học đơn giản.
    
    Args:
        expression: Biểu thức toán học (ví dụ: "2 + 3 * 4")
        
    Returns:
        Kết quả tính toán
    """
    try:
        # Chỉ cho phép các ký tự an toàn
        allowed_chars = set('0123456789+-*/.()')
        if not all(c in allowed_chars or c.isspace() for c in expression):
            return "Lỗi: Biểu thức chứa ký tự không hợp lệ"
            
        result = eval(expression)
        return f"🧮 {expression} = {result}"
    except Exception as e:
        return f"Lỗi tính toán: {str(e)}"

# Danh sách tools
tools = [web_search, calculator]

# Bind tools với LLM
llm_with_tools = llm.bind_tools(tools)

print("✅ Đã định nghĩa các tools:")
for tool in tools:
    print(f"   - {tool.name}: {tool.description}")

## 📊 State Definition

Định nghĩa State để quản lý trạng thái của Agent:

In [ ]:
class AgentState(TypedDict):
    """State chứa messages và metadata của Agent."""
    messages: Annotated[list, add_messages]
    
def print_state(state: AgentState) -> None:
    """Utility function để hiển thị state."""
    print("\n📋 Current State:")
    for i, msg in enumerate(state["messages"]):
        print(f"   {i+1}. {type(msg).__name__}: {msg.content[:100]}...")

## 🤖 Agent Nodes

Tạo các node cho Agent:

In [ ]:
def agent_node(state: AgentState) -> AgentState:
    """Node chính của Agent - quyết định hành động tiếp theo."""
    print("🤖 Agent đang suy nghĩ...")
    
    # Lấy response từ LLM
    response = llm_with_tools.invoke(state["messages"])
    
    # Log agent's decision
    if response.tool_calls:
        tool_names = [call["name"] for call in response.tool_calls]
        print(f"💡 Agent quyết định sử dụng tools: {', '.join(tool_names)}")
    else:
        print("💬 Agent trả lời trực tiếp (không dùng tool)")
    
    return {"messages": [response]}

def should_continue(state: AgentState) -> Literal["tools", "end"]:
    """Conditional edge - quyết định có sử dụng tools hay kết thúc."""
    last_message = state["messages"][-1]
    
    if last_message.tool_calls:
        print("🛠️ Chuyển đến tool execution")
        return "tools"
    else:
        print("🏁 Hoàn thành - không cần tools")
        return "end"

# Tạo tool node
tool_node = ToolNode(tools)

## 🕸️ Xây dựng StateGraph

Tạo graph kết hợp Agent và Tools:

In [ ]:
# Khởi tạo StateGraph
workflow = StateGraph(AgentState)

# Thêm nodes
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# Thêm edges
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)
workflow.add_edge("tools", "agent")

# Compile graph
app = workflow.compile()

print("✅ Đã xây dựng Agent-Tool Graph!")
print("\n🔄 Workflow:")
print("   START → agent → [should_continue] → tools/end")
print("              ↑                         ↓")
print("              └─────────────────────────┘")

## 🧪 Ví dụ thực hành

### Ví dụ 1: Tìm kiếm thông tin trực tuyến

In [ ]:
def run_agent_example(query: str):
    """Chạy agent với một query cụ thể."""
    print(f"\n🚀 Bắt đầu xử lý: '{query}'")
    print("=" * 60)
    
    initial_state = {
        "messages": [HumanMessage(content=query)]
    }
    
    # Chạy agent
    result = app.invoke(initial_state)
    
    print("\n📋 Kết quả cuối cùng:")
    print(result["messages"][-1].content)
    
    return result

# Test với tìm kiếm web
result1 = run_agent_example(
    "Tìm kiếm thông tin về LangGraph và cho tôi biết nó là gì?"
)

### Ví dụ 2: Tính toán

In [ ]:
# Test với calculator
result2 = run_agent_example(
    "Tính giúp tôi (125 + 75) * 3 - 50"
)

### Ví dụ 3: Kết hợp nhiều tools

In [ ]:
# Test kết hợp nhiều tools
result3 = run_agent_example(
    "Tìm kiếm giá Bitcoin hiện tại và tính xem nếu tôi có 0.5 BTC thì bằng bao nhiêu USD?"
)

### Ví dụ 4: Câu hỏi không cần tools

In [ ]:
# Test câu hỏi đơn giản không cần tools
result4 = run_agent_example(
    "Xin chào! Bạn có thể làm gì?"
)

## 🔍 Phân tích quy trình Agent

Hãy phân tích cách Agent hoạt động:

In [ ]:
def analyze_agent_process(query: str, max_steps: int = 10):
    """Phân tích từng bước của Agent."""
    print(f"\n🔬 Phân tích quy trình cho: '{query}'")
    print("=" * 70)
    
    initial_state = {
        "messages": [HumanMessage(content=query)]
    }
    
    step = 0
    current_state = initial_state
    
    # Chạy từng step
    for event in app.stream(initial_state):
        step += 1
        if step > max_steps:
            print("⚠️ Đã đạt giới hạn số bước!")
            break
            
        for node_name, node_output in event.items():
            print(f"\n📍 Bước {step}: Node '{node_name}'")
            
            if node_name == "agent":
                last_msg = node_output["messages"][-1]
                if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                    print(f"   🤖 Agent gọi tools: {[tc['name'] for tc in last_msg.tool_calls]}")
                else:
                    print(f"   💬 Agent phản hồi: {last_msg.content[:100]}...")
                    
            elif node_name == "tools":
                tool_msgs = node_output["messages"]
                for msg in tool_msgs:
                    if isinstance(msg, ToolMessage):
                        print(f"   🛠️ Tool '{msg.name}' kết quả: {msg.content[:100]}...")
    
    print(f"\n✅ Hoàn thành sau {step} bước!")

# Phân tích một ví dụ phức tạp
analyze_agent_process(
    "Tìm thông tin về Python 3.12 và tính xem nếu có 100 developer sử dụng, "
    "mỗi người tiết kiệm 2 giờ/tuần thì tổng cộng tiết kiệm bao nhiêu giờ/năm?"
)

## 📈 Monitoring và Debugging

Thêm khả năng theo dõi hoạt động của Agent:

In [ ]:
class AgentMonitor:
    """Class để theo dõi hoạt động của Agent."""
    
    def __init__(self):
        self.reset_stats()
    
    def reset_stats(self):
        self.stats = {
            "total_steps": 0,
            "agent_calls": 0,
            "tool_calls": 0,
            "tools_used": set(),
            "execution_time": 0
        }
    
    def log_step(self, node_name: str, output: dict):
        """Log một bước thực hiện."""
        self.stats["total_steps"] += 1
        
        if node_name == "agent":
            self.stats["agent_calls"] += 1
            # Kiểm tra tool calls
            last_msg = output["messages"][-1]
            if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                for tc in last_msg.tool_calls:
                    self.stats["tools_used"].add(tc['name'])
                    
        elif node_name == "tools":
            self.stats["tool_calls"] += 1
    
    def print_stats(self):
        """Hiển thị thống kê."""
        print("\n📊 Thống kê Agent:")
        print(f"   • Tổng số bước: {self.stats['total_steps']}")
        print(f"   • Agent calls: {self.stats['agent_calls']}")
        print(f"   • Tool calls: {self.stats['tool_calls']}")
        print(f"   • Tools đã dùng: {', '.join(self.stats['tools_used']) if self.stats['tools_used'] else 'Không có'}")

def run_with_monitoring(query: str):
    """Chạy Agent với monitoring."""
    monitor = AgentMonitor()
    
    print(f"\n🔍 Monitoring query: '{query}'")
    print("=" * 50)
    
    initial_state = {
        "messages": [HumanMessage(content=query)]
    }
    
    # Stream với monitoring
    for event in app.stream(initial_state):
        for node_name, output in event.items():
            monitor.log_step(node_name, output)
    
    monitor.print_stats()
    return monitor.stats

# Test monitoring
stats = run_with_monitoring(
    "Tìm kiếm thông tin về 'Artificial General Intelligence' và tính 2^10"
)

## 🎯 Giải thích & Phân tích

### Cách Agent lựa chọn Tools

Agent sử dụng LLM để quyết định công cụ nào phù hợp dựa trên:

1. **Phân tích ngữ cảnh**: LLM hiểu nội dung câu hỏi
2. **Tool descriptions**: Mô tả chức năng của từng tool
3. **Pattern matching**: Nhận diện mẫu câu hỏi (tìm kiếm, tính toán, etc.)
4. **Multi-step reasoning**: Có thể sử dụng nhiều tools trong một workflow

### Ưu điểm của LangGraph Agent-Tool Architecture:

✅ **Modularity**: Tools độc lập, dễ thêm/bớt  
✅ **Flexibility**: Agent tự quyết định workflow  
✅ **Transparency**: Có thể theo dõi từng bước  
✅ **Error handling**: Xử lý lỗi gracefully  
✅ **Extensibility**: Dễ mở rộng thêm tools mới  

### Thách thức:

⚠️ **Cost**: Mỗi agent call = 1 LLM call  
⚠️ **Latency**: Multi-step có thể chậm  
⚠️ **Reliability**: Phụ thuộc vào chất lượng LLM  
⚠️ **Tool selection**: Có thể chọn sai tool  

## 🔧 Nâng cao: Custom Tool với Error Handling

In [ ]:
@tool
def file_analyzer(file_path: str) -> str:
    """Phân tích file text và đưa ra thống kê cơ bản.
    
    Args:
        file_path: Đường dẫn đến file
        
    Returns:
        Thống kê về file (số dòng, từ, ký tự)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        lines = len(content.split('\n'))
        words = len(content.split())
        chars = len(content)
        
        return f"""📄 Phân tích file '{file_path}':
   • Số dòng: {lines:,}
   • Số từ: {words:,}
   • Số ký tự: {chars:,}
   • Kích thước: {chars/1024:.1f} KB"""
        
    except FileNotFoundError:
        return f"❌ Không tìm thấy file: {file_path}"
    except PermissionError:
        return f"❌ Không có quyền đọc file: {file_path}"
    except Exception as e:
        return f"❌ Lỗi khi đọc file: {str(e)}"

# Test tool mới
print("🧪 Test file analyzer tool:")
result = file_analyzer("/etc/hosts")  # File thường tồn tại trên Linux/Mac
print(result)

## 🏆 Kết luận & Gợi ý mở rộng

### Những gì chúng ta đã học:

1. **Agent Architecture**: Cách xây dựng agent với khả năng sử dụng tools
2. **Tool Integration**: Tích hợp và quản lý nhiều tools khác nhau
3. **Conditional Logic**: Sử dụng conditional edges để điều khiển workflow
4. **Monitoring**: Theo dõi và debug hoạt động của agent
5. **Error Handling**: Xử lý lỗi trong tools một cách graceful

### 🚀 Gợi ý mở rộng:

1. **Thêm tools nâng cao**:
   - Database query tool
   - Email sending tool  
   - Image processing tool
   - API integration tools

2. **Memory & Context**:
   - Lưu trữ conversation history
   - Context-aware tool selection
   - Learning from previous interactions

3. **Multi-agent systems**:
   - Specialized agents cho different domains
   - Agent collaboration
   - Hierarchical agent structures

4. **Performance optimization**:
   - Tool caching
   - Parallel tool execution
   - Smart tool selection

5. **Advanced monitoring**:
   - LangSmith integration
   - Performance metrics
   - Success rate tracking

### 📚 Tài liệu tham khảo:

- [LangGraph Agent Documentation](https://python.langgraph.org/tutorials/introduction/)
- [LangChain Tools](https://python.langchain.com/docs/modules/tools/)
- [Agent Patterns](https://python.langgraph.org/concepts/agent_architectures/)

---

**Tiếp theo**: Part 4 - Multi-Agent Systems và Advanced Workflows! 🎯